In [2635]:
import cplex
from cplex.exceptions import CplexError
import sys
import pandas as pd
import numpy as np
import docplex

In [2636]:
def pprint(constraints: list, names: list):
    for c in constraints:
        _c = [str(_i) + "*" + n if _i != 0 else "0" for _i, n in zip(c[0], names)]
        print(c[3] + ": ( [" + " ".join(_c) + "]  " + c[1] + "  " + str(c[2]) + " )") 
        

In [2637]:
xls = pd.ExcelFile('logistic.xlsx')

vehicles = [1,2,3,4,5]                                # V
customers = [1,2,3,4,5,6,7,8,9]                       # C

num_vehicles = 5
num_customers = 7
num_locations = num_customers + 1                     
num_names = (num_locations ** 2) * num_vehicles

locations = [0] + customers[:num_customers]           # C': gồm các customers và Depot suất phát + Depot kết thúc

# quy ước: 
#    + (i,j) là cạnh (edge) thuộc tập E
#    + i,j là 2 customer liền kề trên 1 route

In [2638]:
my_names = ["x"+str(i)+str(j) + str(k) for k in range(1,num_vehicles + 1) for i in range(0,num_locations) for j in range(0, num_locations) ]  

print(len(my_names))
print("Names ", my_names)
def display(constraints):
    pprint(constraints, my_names)

320
Names  ['x001', 'x011', 'x021', 'x031', 'x041', 'x051', 'x061', 'x071', 'x101', 'x111', 'x121', 'x131', 'x141', 'x151', 'x161', 'x171', 'x201', 'x211', 'x221', 'x231', 'x241', 'x251', 'x261', 'x271', 'x301', 'x311', 'x321', 'x331', 'x341', 'x351', 'x361', 'x371', 'x401', 'x411', 'x421', 'x431', 'x441', 'x451', 'x461', 'x471', 'x501', 'x511', 'x521', 'x531', 'x541', 'x551', 'x561', 'x571', 'x601', 'x611', 'x621', 'x631', 'x641', 'x651', 'x661', 'x671', 'x701', 'x711', 'x721', 'x731', 'x741', 'x751', 'x761', 'x771', 'x002', 'x012', 'x022', 'x032', 'x042', 'x052', 'x062', 'x072', 'x102', 'x112', 'x122', 'x132', 'x142', 'x152', 'x162', 'x172', 'x202', 'x212', 'x222', 'x232', 'x242', 'x252', 'x262', 'x272', 'x302', 'x312', 'x322', 'x332', 'x342', 'x352', 'x362', 'x372', 'x402', 'x412', 'x422', 'x432', 'x442', 'x452', 'x462', 'x472', 'x502', 'x512', 'x522', 'x532', 'x542', 'x552', 'x562', 'x572', 'x602', 'x612', 'x622', 'x632', 'x642', 'x652', 'x662', 'x672', 'x702', 'x712', 'x722', 'x73

In [2639]:
cost_matrix = pd.read_excel(xls, 'Cost Matrix')
my_obj = [int(cost_matrix.iat[i,j]) for i in range(0,num_locations ) for j in range(1,num_locations + 1)]*num_vehicles

print("Object ",my_obj)

Object  [0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 18, 22, 13, 30, 23, 0, 32, 29, 18, 10, 26, 28, 10, 32, 0, 2, 20, 31, 17, 27, 4, 29, 2, 0, 19, 28, 12, 8, 18, 18, 20, 19, 0, 5, 8, 18, 22, 10, 31, 28, 5, 0, 17, 5, 13, 26, 17, 12, 8, 17, 0, 0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 18, 22, 13, 30, 23, 0, 32, 29, 18, 10, 26, 28, 10, 32, 0, 2, 20, 31, 17, 27, 4, 29, 2, 0, 19, 28, 12, 8, 18, 18, 20, 19, 0, 5, 8, 18, 22, 10, 31, 28, 5, 0, 17, 5, 13, 26, 17, 12, 8, 17, 0, 0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 18, 22, 13, 30, 23, 0, 32, 29, 18, 10, 26, 28, 10, 32, 0, 2, 20, 31, 17, 27, 4, 29, 2, 0, 19, 28, 12, 8, 18, 18, 20, 19, 0, 5, 8, 18, 22, 10, 31, 28, 5, 0, 17, 5, 13, 26, 17, 12, 8, 17, 0, 0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 18, 22, 13, 30, 23, 0, 32, 29, 18, 10, 26, 28, 10, 32, 0, 2, 20, 31, 17, 27, 4, 29, 2, 0, 19, 28, 12, 8, 18, 18, 20, 19, 0, 5, 8, 18, 22, 10, 31, 28, 5, 0, 17, 5, 13, 26, 17, 12, 8, 17, 0, 0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 

In [2640]:
capacity_matrix = pd.read_excel(xls, 'Capicity')
vehicles_capacity = [capacity_matrix.iat[i,1] for i in range(0, num_vehicles)]
vehicles_capacity

[100, 70, 145, 90, 50]

In [2641]:
demand_matrix = pd.read_excel(xls, 'Demand Matrix')
demand = {customers[i]: demand_matrix.iat[i,1] for i in range(0, num_customers)}
demand

{1: 32, 2: 45, 3: 16, 4: 15, 5: 22, 6: 50, 7: 33}

In [2642]:
lower_bounds = [0]*num_names
upper_bounds = [1]*num_names

print("Lb ", lower_bounds)
print("Rb ", upper_bounds)

Lb  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Rb  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [2643]:
# CONSTRAINTS_2: 
# # Một cạnh bất kỳ (i->j) chỉ tổn tại trên duy nhất 1 Vehicle. Nhưng i CHỈ LÀ customers, j là locations
contrains_2 = []
_i = 0
for i in customers[:num_customers]:
    con_ = [(1 if int(name[1]) != int(name[2])  and int(name[1]) == i else 0) for name in my_names]
    if len([c for c in con_  if c != 0]) > 0:
        _i += 1
        contrains_2.append((con_,"E", 1, "C2" + str(_i)))
    
display(contrains_2)


C21: ( [0 0 0 0 0 0 0 0 1*x101 0 1*x121 1*x131 1*x141 1*x151 1*x161 1*x171 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x102 0 1*x122 1*x132 1*x142 1*x152 1*x162 1*x172 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x103 0 1*x123 1*x133 1*x143 1*x153 1*x163 1*x173 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x104 0 1*x124 1*x134 1*x144 1*x154 1*x164 1*x174 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x105 0 1*x125 1*x135 1*x145 1*x155 1*x165 1*x175 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C22: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x201 1*x211 0 1*x231 1*x241 1*x251 1*x261 1*x271 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [2644]:
# CONSTRAINTS_3: 
# # Lượng hàng chở đển Customer của 1 Vehicle phải nhỏ hơn bằng Capacity. Nhưng i CHỈ LÀ customers, j là locations
contrains_3 = []
val = lambda v, c, n1, n2, n3: 0 if n3 != v else -1*int(demand[n1]) if int(n1) != 0 and n1 != n2 else c if n1 == 0 and n2 != 0 else 0
_i = 0
for veh, cap in zip(vehicles[0:num_vehicles],vehicles_capacity[0:num_vehicles]): 
    _i += 1
    contrains_3.append(([val(veh, float(cap), int(name[1]), int(name[2]), int(name[3])) for name in my_names],"G", 0.0, "C3" + str(_i)))
display(contrains_3)

C31: ( [0 100.0*x011 100.0*x021 100.0*x031 100.0*x041 100.0*x051 100.0*x061 100.0*x071 -32*x101 0 -32*x121 -32*x131 -32*x141 -32*x151 -32*x161 -32*x171 -45*x201 -45*x211 0 -45*x231 -45*x241 -45*x251 -45*x261 -45*x271 -16*x301 -16*x311 -16*x321 0 -16*x341 -16*x351 -16*x361 -16*x371 -15*x401 -15*x411 -15*x421 -15*x431 0 -15*x451 -15*x461 -15*x471 -22*x501 -22*x511 -22*x521 -22*x531 -22*x541 0 -22*x561 -22*x571 -50*x601 -50*x611 -50*x621 -50*x631 -50*x641 -50*x651 0 -50*x671 -33*x701 -33*x711 -33*x721 -33*x731 -33*x741 -33*x751 -33*x761 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [2645]:
# CONSTRAINTS_4: 
# # Với mỗi Vehicle, khi suất phát từ Depot, nó chỉ có thể đến duy nhất 1 Customer
contrains_4 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    _i += 1
    contrains_4.append(([(1 if (int(name[1]) == 0 and int(name[3]) == k and int(name[2]) != 0) else 0) for name in my_names],"L", 1, "C4" + str(_i)))
    
display(contrains_4)

C41: ( [0 1*x011 1*x021 1*x031 1*x041 1*x051 1*x061 1*x071 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  L  1 )
C42: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x012 1*x022 1*x032 1*x042 1*x052 1*x062 1*x072 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [2646]:
# CONSTRAINTS_5: 
# # Trên route của Vehicle k có cạnh (i->j) : tổng tất cả cạnh đầu 'ra' của i BẰNG tổng tất cả cạnh đầu 'vào' của j
contrains_5 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in locations[:num_locations]: # i
        _i += 1
        foo = lambda k,p,n: 0 if int(n[3]) != k or int(n[2]) == int(n[1]) else 1 if int(n[1]) == p else -1 if int(n[2]) == p else 0                
        contrains_5.append(([foo(k, p, name) for name in my_names],"E", 0, "C5" + str(_i)))

display(contrains_5)

C51: ( [0 1*x011 1*x021 1*x031 1*x041 1*x051 1*x061 1*x071 -1*x101 0 0 0 0 0 0 0 -1*x201 0 0 0 0 0 0 0 -1*x301 0 0 0 0 0 0 0 -1*x401 0 0 0 0 0 0 0 -1*x501 0 0 0 0 0 0 0 -1*x601 0 0 0 0 0 0 0 -1*x701 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  0 )
C52: ( [0 -1*x011 0 0 0 0 0 0 1*x101 0 1*x121 1*x131 1*x141 1*x151 1*x161 1*x171 0 -1*x211 0 0 0 0 0 0 0 -1*x311 0 0 0 0 0 0 0 -1*x411 0 0 0 0 0 0 0 -1*x511 0 0 0 0 0 0 0 -1*x611 0 0 0 0 0 0 0 -1*x711 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [2647]:
# CONSTRAINTS_6: 
# # Route của Vehicle k phải luôn trở về Depot (n+1), tức là luôn tồn tại điểm i trên route của k sao cho nó đi đến n+1
# # trong đó n+1 là Depot kết thúc
contrains_6 = []
_i = 0
for veh in vehicles[0:num_vehicles]: 
    _i += 1
    contrains_6.append(([(1 if (int(name[3]) == veh and int(name[2]) == 0 and int(name[1]) != 0) else 0) for name in my_names],"L", 1, "C6" + str(_i)))
    
display(contrains_6)


C61: ( [0 0 0 0 0 0 0 0 1*x101 0 0 0 0 0 0 0 1*x201 0 0 0 0 0 0 0 1*x301 0 0 0 0 0 0 0 1*x401 0 0 0 0 0 0 0 1*x501 0 0 0 0 0 0 0 1*x601 0 0 0 0 0 0 0 1*x701 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  L  1 )
C62: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x102 0 0 0 0 0 0 0 1*x202 0 0 0 0 0 0 0 1*x302 0 0 0 0 0 0 0 1*x402 0 0 0 0 0 0 0 1*x502 0 0 0 0 0 0 0 1*x602 0 0 0 0 0 0 0 1*x702 0 0 0 0 0 0 0 0 0 0 0 0

In [2648]:
# CONSTRAINTS_X: vehicles ko đc đứng tại chỗ
contrains_x = []
_i = 0

_i += 1
contrains_x.append(([(1 if int(name[1]) == int(name[2]) else 0) for name in my_names],"E", 0, "CX" + str(_i)))

display(contrains_x)


CX1: ( [1*x001 0 0 0 0 0 0 0 0 1*x111 0 0 0 0 0 0 0 0 1*x221 0 0 0 0 0 0 0 0 1*x331 0 0 0 0 0 0 0 0 1*x441 0 0 0 0 0 0 0 0 1*x551 0 0 0 0 0 0 0 0 1*x661 0 0 0 0 0 0 0 0 1*x771 1*x002 0 0 0 0 0 0 0 0 1*x112 0 0 0 0 0 0 0 0 1*x222 0 0 0 0 0 0 0 0 1*x332 0 0 0 0 0 0 0 0 1*x442 0 0 0 0 0 0 0 0 1*x552 0 0 0 0 0 0 0 0 1*x662 0 0 0 0 0 0 0 0 1*x772 1*x003 0 0 0 0 0 0 0 0 1*x113 0 0 0 0 0 0 0 0 1*x223 0 0 0 0 0 0 0 0 1*x333 0 0 0 0 0 0 0 0 1*x443 0 0 0 0 0 0 0 0 1*x553 0 0 0 0 0 0 0 0 1*x663 0 0 0 0 0 0 0 0 1*x773 1*x004 0 0 0 0 0 0 0 0 1*x114 0 0 0 0 0 0 0 0 1*x224 0 0 0 0 0 0 0 0 1*x334 0 0 0 0 0 0 0 0 1*x444 0 0 0 0 0 0 0 0 1*x554 0 0 0 0 0 0 0 0 1*x664 0 0 0 0 0 0 0 0 1*x774 1*x005 0 0 0 0 0 0 0 0 1*x115 0 0 0 0 0 0 0 0 1*x225 0 0 0 0 0 0 0 0 1*x335 0 0 0 0 0 0 0 0 1*x445 0 0 0 0 0 0 0 0 1*x555 0 0 0 0 0 0 0 0 1*x665 0 0 0 0 0 0 0 0 1*x775]  E  0 )


In [2649]:
# CONSTRAINTS_Y: vehicles ko đc quay ngược lại
contrains_y = []
_i = 0

for i in customers[:num_customers]:
    for j in customers[:num_customers]:
        if i != j:
            _i += 1
            contrains_y.append(([(1 if ((int(name[1]) == i and j == int(name[2])) or (int(name[1]) == j and i == int(name[2]))) else 0) for name in my_names],"L", 1, "CX" + str(_i)))

display(contrains_y)


CX1: ( [0 0 0 0 0 0 0 0 0 0 1*x121 0 0 0 0 0 0 1*x211 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x122 0 0 0 0 0 0 1*x212 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x123 0 0 0 0 0 0 1*x213 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x124 0 0 0 0 0 0 1*x214 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x125 0 0 0 0 0 0 1*x215 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  L  1 )
CX2: ( [0 0 0 0 0 0 0 0 0 0 0 1*x131 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x311 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x132 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x312 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

CX20: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x241 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x421 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x242 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x422 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x243 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x423 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x244 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x424 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x245 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x425 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  L  1 )
CX21: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x341 0 0 0 0 0 0 1*x431 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x342 0 0 0 0 0 0 1*x432 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [2650]:
# # CONSTRAINT_Y
# _i = 0
# for 
# contrains_x.append(([(1 if int(name[1]) == int(name[2]) else 0) for name in my_names],"E", 0, "CX" + str(_i)))

In [2651]:
# Merge segment constraints
constraints_group = []
constraints_group += contrains_2 
constraints_group += contrains_3 
constraints_group += contrains_4 
constraints_group += contrains_5 
constraints_group += contrains_6
constraints_group += contrains_x
constraints_group += contrains_y



# Convert for cplex
constraints_ = []
constraint_senses = []
rhs = []
constraint_names = []
for i in constraints_group:
    constraints_.append([my_names, i[0]])
    constraint_senses.append(i[1])
    rhs.append(i[2])
    constraint_names.append(i[3])
    
    
# print(constraints_)
# print(constraint_senses)
# print(rhs)

In [2652]:
# Create Cplex Problems
myProblem = None

try:
    myProblem = cplex.Cplex()     
except CplexError:
    print ("exc")


myProblem.variables.add(obj = my_obj,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = my_names)

myProblem.objective.set_sense(myProblem.objective.sense.minimize)

for i in range(0, num_names):
    myProblem.variables.set_types(i,myProblem.variables.type.binary)
    
myProblem.linear_constraints.add(lin_expr = constraints_,
                                 senses = constraint_senses,
                                 rhs = rhs,
                                 names = constraint_names)

range(0, 105)

In [2653]:
# Find the solutions
try:
    myProblem.solve()
    
    print(myProblem.get_stats())
    values=myProblem.solution.get_values()
    names=myProblem.variables.get_names()
    
    for v, n in zip(values, names):
        if v != 0:
            print(n, " - ",v)
except cplex.exceptions.errors.CplexSolverError:
    print("No solution")

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 22 rows and 42 columns.
MIP Presolve modified 280 coefficients.
Reduced MIP has 83 rows, 278 columns, and 1320 nonzeros.
Reduced MIP has 278 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.21 ticks)
Found incumbent of value 201.000000 after 0.01 sec. (2.72 ticks)
Probing time = 0.00 sec. (1.66 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 83 rows, 278 columns, and 1320 nonzeros.
Reduced MIP has 278 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.86 ticks)
Probing time = 0.02 sec. (1.65 ticks)
Clique table members: 766.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (1.14 ticks)

        Nodes                              